Pineapple - Robot Shop Pattern 1 

In [ ]:
# Required Imports 
import pandas as pd 
import numpy as np 
import os 
import matplotlib.pyplot as plt
import json
import datetime, time
from matplotlib.dates import date2num
from scipy.interpolate import make_interp_spline
import seaborn as sns

In [ ]:
# https://stackoverflow.com/a/1879011/14626757
# Returns a walked path under collected_data
def f(starting_path):
  alld = {'': {}}

  for dirpath, dirnames, filenames in os.walk(starting_path):
    d = alld
    dirpath = dirpath[len(starting_path):]
    for subd in dirpath.split(os.sep):
      based = d
      d = d[subd]
    if dirnames:
      for dn in dirnames:
        d[dn] = {}
    else:
      based[subd] = filenames
  return alld['']

In [ ]:
# standard code to convert data to a timestamp, metric list 
def convert_enddate_to_seconds(ts):
    """Takes ISO 8601 format(string) and converts into epoch time."""
    dt = datetime.datetime.strptime(ts[:-7],'%Y-%m-%dT%H:%M:%S')+\
                datetime.timedelta(hours=int(ts[-5:-3]),
                minutes=int(ts[-2:]))*int(ts[-6:-5]+'1')
    seconds = time.mktime(dt.timetuple()) + dt.microsecond/1000000.0
    return seconds

# returns 2 lists for average values, and timestamp
def get_list(timeseries):
    y = [val['average'] for val in timeseries]
    x = [pd.to_datetime(convert_enddate_to_seconds(val['timeStamp']),unit='s') for val in timeseries]
    return x,y

# 
def return_time_metric(filename):
    data = []
    with open(filename) as f: 
        lines = f.readlines()
        for l in lines:
            # 2022-02-22 20:55:00+00:00: 11.32125
            d,t,v = l.split()
            data.append(float(v))
    return data
    # x, y = get_list(timeseries)
    # return x,y

def create_graph(x,y,):
    # def clean_plot(data, metric):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    
    ax.plot(x, y)   # 2021-12-31T18:36:00+00:00 2021-12-31T19:05:00+00:00


In [ ]:
# Generate file listings
filepath = "../collected_data/lime"
api      = os.path.join(filepath, "api")
machine  = os.path.join(filepath, "machine")

api_listing     = f(api)
machine_listing = f(machine)

In [ ]:
big_machine = {}
# for size, iteration in machine_listing.items():
#     metrics   = []
#     timestamp = 0
#     for iteration, files in iteration.items(): 
#         temp_path = os.path.join(machine, size, iteration)
#         timestamp, metric = return_time_metric(os.path.join(temp_path, "percentageCPU.txt"))
#         metrics.append(metric)
#     tiny_machine = {}

#     for i in range(0, len(timestamp)):
#         metric_per_time = []
#         for n in metrics:
#             metric_per_time.append(n[i])
#         metric_per_time = [float(x) for x in metric_per_time if x != None]
#         tiny_machine[timestamp[i]] = np.mean(metric_per_time) #Change here to maintain value or not.
#     big_machine[size] = tiny_machine

for size, iteration in machine_listing.items():
    metric   = []
    timestamp = 0
    for iteration, files in iteration.items(): 
        temp_path = os.path.join(machine, size, iteration)
        metric = return_time_metric(os.path.join(temp_path, "percentageCPU.txt"))
    fig = plt.figure(figsize =(11, 7))
    ax = fig.add_subplot(111)
    plt.boxplot(np.array(metric)) 
    plt.ylim(0, 100)
    plt.show()

# # Sort big_machine basedo n list 
# machine_names = ["Standard_A2_v2","Standard_A4_v2","Standard_A8_v2","Standard_B12ms","Standard_B16ms"]
# ordered_dict = {}
# for item in machine_names:
#     ordered_dict[item] = big_machine[item]

# def do_this(machine_dict):
#     ranged = []
#     for key, value in machine_dict.items(): 
#         ranged.append([x for x in list(value.values()) if str(x) != 'nan'])
  
#     fig = plt.figure(figsize =(11, 7))
#     ax = fig.add_subplot(111)
#     plt.boxplot( ranged) 
#     ax.set_xticklabels(list(machine_dict.keys()))
#     plt.ylim(0, 100)
#     plt.show()

# do_this(ordered_dict)

ValueError: could not convert string to float: 'None'

In [ ]:
big_api = {}
# Get all values
for size, iteration in api_listing.items():
    # df_files = {} 
    df_list = list()
    for iteration, files in iteration.items(): 
        temp_path = os.path.join(api, size, iteration)
        file_name = "load_stats_history.csv"
        df = pd.read_csv(os.path.join(temp_path, file_name))
        df['Timestamp']= pd.to_datetime(df['Timestamp'] ,unit='s')
        # df_files[iteration] = df
        df_list.append(df)
        
    big_api[size] =  df_list

# average metrics
for key, values in big_api.items():
    convert_to_df = {}
    for i in range(0, len(values)):
        
        timestamp =  values[i]["Timestamp"]
        plt.plot(timestamp, values[i]["Total Average Response Time"]) 
        plt.title(key)
        plt.xticks(rotation=45)
    plt.show()




AttributeError: 'list' object has no attribute 'items'

In [ ]:
# Latency Average Percentages
big_api = {}
# Get all values
fig = plt.figure(figsize =(11, 7))
ax = fig.add_subplot(111)
machine_names = ["standarda2v2","standarda4v2","standarda8v2","standardb12ms","standardb16ms"]
ordered_listing = {}
for item in machine_names:
    ordered_listing[item] = api_listing[item]

for size, iteration in ordered_listing.items():
    # df_files = {} 

    df_list = list()
    new_df = pd.DataFrame(columns=["Requests/s", "50%","66%","75%","80%","90%","95%","98%","99%","99.9%","99.99%","100%"])

    for iteration, files in iteration.items(): 
        temp_path = os.path.join(api, size, iteration)
        file_name = "load_stats_history.csv"
        df = pd.read_csv(os.path.join(temp_path, file_name))
        df["datetime"] = df["Timestamp"].apply(lambda t: datetime.fromtimestamp(t))
        requests = df[["Requests/s", "50%","66%","75%","80%","90%","95%","98%","99%","99.9%","99.99%","100%"]]
        # stats_history_df = stats_history_df[stats_history_df.Name != "Aggregated"]
        requests = requests[requests["Requests/s"] != 0.0]
        print(requests)
        new_df = new_df.append(requests, ignore_index=True )

        # print(requests)
        # df_list
    print(size)
    plt.plot(list(new_df.mean().to_dict().keys())[1::], list(new_df.mean().to_dict().values())[1::])

ax.legend(list(ordered_listing.keys()))
ax.set_xticklabels(list(requests.keys())[1::])
plt.xticks(rotation=45)

plt.show()
        # df_files[iteration] = df
#         df_list.append(df)
        
#     big_api[size] =  df_list

# # average metrics
# for key, values in big_api.items():
#     convert_to_df = {}
#     for i in range(0, len(values)):
        
#         timestamp =  values[i]["Timestamp"]
#         plt.plot(timestamp, values[i]["Total Average Response Time"]) 
#         plt.title(key)
#         plt.xticks(rotation=45)
#     plt.show()


